# 2 決定分析の基本

In [1]:
import pandas as pd
import numpy as np

# DataFrame の全角文字の出力をきれいにする
pd.set_option('display.unicode.east_asian_width', True)

## 2.1 決定分析の初歩　〜 2.3 決定分析における Python の利用

- ケース
  - ある工場である製品を作っている
  - 工場には製品を作るための機械が2台ある
    - 機械を多く動かせば動かすほどたくさんの製品が作れるが、動かす機械の台数が増えると稼働費用が増える
    - 機械をまったく稼働させない場合も、工場を維持する固定費は発生
  - 製品の需要量は月ごとに変動
    - 需要量を超える製品を作っても、売れないので収入は増えない
    - 売れ残った製品は月末にすべて廃棄されるのでムダになる
  - 機械の稼働台数は1ヶ月おきに見直す
- モデル化
  - 選択肢: 機械の稼働台数（0台、1台、2台）
    - $a_j ∈ A=\{0, 1, 2\}$
    - （ $a$ は Action の頭文字からとっている）
  - 自然の状態: 製品の需要量（好況、不況）
    - $θ_i ∈ Θ=\{好況, 不況\}$
  - 結果（利得）
    - $c(a_j, θ_i)$
    - （ $c$ は Consequence の頭文字からとっている）
  - 意思決定問題
    - $D = \{A, Θ, c\}$

具体的に以下のような利得行列が得られたとする

| | 0台稼働 $a_1$ | 1台稼働 $a_2$ | 2台稼働 $a_3$ |
| :---: | :---: | :---: | :---: |
| 好況 $θ_1$ | $c(a_1, θ_1) = -100$ | $c(a_2, θ_1) = 300$ | $c(a_3, θ_1) = 700$ | 
| 好況 $θ_2$ | $c(a_1, θ_2) = -100$ | $c(a_2, θ_2) = 300$ | $c(a_3, θ_2) = -300$ |

この場合、好況・不況関係なく、0台稼働よりも1台稼働のほうが利得が高い
→ 1台稼働を選択することは、0台稼働を「優越」する

### 決定基準
※ 伝統的にしばしば用いられ、意味付けが明確なもの5種類

| 基準名 | 概要 | 数式表現 | 何台稼働がベスト？ |
| :---: | :--- | :---: | :---: |
| Maximax | 最大利得が発生する自然の状態を仮定したうえで、そのときの利得を最大にするように選択肢を選ぶ。<br>**最も楽観的な決定基準** | $ a^{\text{max}} = \text{argmax}_{a_j} \lambda_{\text{max}}(a_j) $ <br> $ \lambda_{\text{max}}(a_j) = \text{max}_{\theta_i} c(a_j, \theta_i) $ | 2 |
| Maximin | 最小利得が発生する自然の状態を仮定したうえで、そのときの利得を最大にするように選択肢を選ぶ。<br>**最も悲観的な決定基準** | $ a^{\text{min}} = \text{argmax}_{a_j} \lambda_{\text{min}}(a_j) $ <br> $ \lambda_{\text{min}}(a_j) = \text{min}_{\theta_i} c(a_j, \theta_i) $ | 1 |
| Hurwicz（ハーヴィッツ） | 楽観係数と呼ばれるハイパーパラメータによってMaximax, Maximinの重みを変える。<br>**極めて主観的な決定基準** | $ a^{\text{Hurwicz}} = \text{argmax}_{a_j} \lambda_{\text{Hurwicz}}(a_j) $ <br> $ \lambda_{\text{Hurwicz}}(a_j) = \alpha \cdot \text{max}_{\theta_i}[c(a_j, \theta_i)] + (1 - \alpha) \cdot \text{min}_{\theta_i}[c(a_j, \theta_i)] $ | 1でも2でもどちらでもよい |
| Minimax regret<br>= Minimax<br>= Savage | 「自然の状態ごとに見た最大利得と比較した差分（=リグレット）の最大値」を最小にする | $ a^{\text{regret}} = \text{argmin}_{a_j} \lambda_{\text{max\_regret}}(a_j) $ <br> $ \lambda_{\text{max\_regret}}(a_j) = \text{max}_{\theta_i} r(a_j, \theta_i)$ <br> $ r(a_j, \theta_i) = \text{max}_{a_j}[c(a_j, \theta_i)] - c(a_j, \theta_i) $| 1 |
| Laplace（ラプラス） | 自然の状態がすべて等しい確率で出現すると考えて、利得の期待値を最大にするように選択肢を選ぶ | $ a^{\text{Laplace}} = \text{argmax}_{a_j} \lambda_{\text{Laplace}}(a_j)$ <br> $ \lambda_{\text{Laplace}}(a_j) = \frac{1}{\# Θ} \Sigma_{i ∈ \# Θ} c(a_j, \theta_i) $ | 1 |

### やってみる

#### 準備

##### 売上行列

In [2]:
# 利得を計算する際のパラメータ
# =========================================
# 以下、単位はすべて「万円」
FIXED_COST = 100 # 工場の固定費用
RUN_COST = 600 # 機械1台の稼働コスト
SALE_PRICE = 0.2 # 製品1つの販売価格

# 以下、単位はすべて「個」
MACHINE_ABILITY = 5_000 # 機械1台で作られる製品数
DEMAND_BOOM = 10_000 # 好況時の需要量
DEMAND_SLUMP = 5_000 # 不況時の需要量
# =========================================

# 出荷される製品の個数
num_product_df = pd.DataFrame({
    "0台": [
        0,
        0
    ],
    "1台": [
        min([MACHINE_ABILITY, DEMAND_BOOM]),
        min([MACHINE_ABILITY, DEMAND_SLUMP]),
    ],
    "2台": [
        min([2 * MACHINE_ABILITY, DEMAND_BOOM]),
        min([2 * MACHINE_ABILITY, DEMAND_SLUMP])
    ]
})
num_product_df.index = ["好況", "不況"]

# 売上行列
sales_df = num_product_df * SALE_PRICE
sales_df

,0台,1台,2台
好況,0.0,1000.0,2000.0
不況,0.0,1000.0,1000.0


##### 製造コスト

In [3]:
run_cost_df = pd.DataFrame({
    "0台": np.repeat(FIXED_COST, 2),
    "1台": np.repeat(FIXED_COST + RUN_COST, 2),
    "2台": np.repeat(FIXED_COST + RUN_COST * 2, 2),
})
run_cost_df.index = ["好況", "不況"]
run_cost_df

,0台,1台,2台
好況,100,700,1300
不況,100,700,1300


##### 利得行列

In [4]:
payoff_df = sales_df - run_cost_df
payoff_df

,0台,1台,2台
好況,-100.0,300.0,700.0
不況,-100.0,300.0,-300.0


#### 各基準の計算

In [5]:
def calc_payoff_table(fixed_cost=100, run_cost=600, sale_price=0.2, machine_ability=5_000, demand_boom=10_000, demand_slump=5_000):
    # 出荷される製品の個数
    num_product_df = pd.DataFrame({
        "0台": [
            0,
            0
        ],
        "1台": [
            min([machine_ability, demand_boom]),
            min([machine_ability, demand_slump]),
        ],
        "2台": [
            min([2 * machine_ability, demand_boom]),
            min([2 * machine_ability, demand_slump])
        ]
    })
    num_product_df.index = ["好況", "不況"]

    # 売上行列
    sales_df = num_product_df * sale_price

    # 製造コスト
    run_cost_df = pd.DataFrame({
        "0台": np.repeat(fixed_cost, 2),
        "1台": np.repeat(fixed_cost + run_cost, 2),
        "2台": np.repeat(fixed_cost + run_cost * 2, 2),
    })
    run_cost_df.index = ["好況", "不況"]

    # 利得行列
    payoff_df = sales_df - run_cost_df
    return payoff_df

payoff_df = calc_payoff_table()
payoff_df

,0台,1台,2台
好況,-100.0,300.0,700.0
不況,-100.0,300.0,-300.0


##### Maximax, Maximin

In [6]:
# idxmax だと最大値が複数ある場合、最初に見つかったものだけを返すため、このような関数を作成している
# ntoe.nkmk.me にも同じようにしろと記載あり: https://note.nkmk.me/python-pandas-idxmax-idxmin/

def get_argmax_list(series):
    return list((series[series == series.max()].index))

print(f"Maximax: {get_argmax_list(payoff_df.max())}")
print(f"Maximin: {get_argmax_list(payoff_df.min())}")

Maximax: ['2台']
Maximin: ['1台']


##### Hurwicz

In [7]:
def hurwicz(payoff_df, alpha=0.6): # alpha は楽観係数
    return payoff_df.max() * alpha + payoff_df.min() * (1 - alpha)

print(f"Hurwicz (alpha=0.6): {get_argmax_list(hurwicz(payoff_df))}")
print(f"Hurwicz (alpha=0.7): {get_argmax_list(hurwicz(payoff_df, alpha=0.7))}")

Hurwicz (alpha=0.6): ['1台', '2台']
Hurwicz (alpha=0.7): ['2台']


##### Minimax Regret

In [8]:
def get_argmin_list(series):
    return list((series[series == series.min()].index))

best_df = pd.concat(
    [payoff_df.max(axis=1)] * payoff_df.shape[1], axis=1
)
best_df.columns = payoff_df.columns
best_df

,0台,1台,2台
好況,700.0,700.0,700.0
不況,300.0,300.0,300.0


In [9]:
regret_df = best_df - payoff_df
regret_df

,0台,1台,2台
好況,800.0,400.0,0.0
不況,400.0,0.0,600.0


In [10]:
print(f"Minimax regret: {get_argmin_list(regret_df.max())}")

Minimax regret: ['1台']


##### Laplace

In [11]:
print(f"Laplace: {get_argmax_list(payoff_df.mean())}")

Laplace: ['1台']


### 感度分析

- 感度分析: 「モデルの前提となった数値の変化が、意思決定の結果にどれほど影響を与えるか」を調べる作業
  - 前提となった数値が少し変わっただけで意思決定の結果が大きく変わるようなら、決定基準から得られた結果を採択するのには慎重になる必要がある
- 以下では、機械1台のコストのみ変動させた場合の Minimax regret で感度分析を試してみる

In [12]:
def minimax_regret(payoff_df):
    best_df = pd.concat(
        [payoff_df.max(axis=1)] * payoff_df.shape[1], axis=1
    )
    best_df.columns = payoff_df.columns
    regret_df = best_df - payoff_df
    return regret_df.max()

In [13]:
payoff2 = calc_payoff_table(run_cost=625) # 機械1台の稼働コストを +25万円
payoff3 = calc_payoff_table(run_cost=575) # 機械1台の稼働コストを -25万円
payoff4 = calc_payoff_table(run_cost=500) # 機械1台の稼働コストを -100万円

print(f"Minimax regret (run_cost=625): {get_argmin_list(minimax_regret(payoff2))}")
print(f"Minimax regret (run_cost=575): {get_argmin_list(minimax_regret(payoff3))}")
print(f"Minimax regret (run_cost=500): {get_argmin_list(minimax_regret(payoff4))}")

Minimax regret (run_cost=625): ['1台']
Minimax regret (run_cost=575): ['1台']
Minimax regret (run_cost=500): ['1台', '2台']


すなわち、今回の例では1台あたりの稼働コストに100万円ほど想定誤差があると、意思決定の結果が変わってしまう